In [8]:
from glob import glob
import os
import xarray as xr
import cftime
import numpy as np
import time 
import datetime

In [9]:
def set_bnds_as_coords(ds):
    new_coords_vars = [var for var in ds.data_vars if 'bnds' in var or 'bounds' in var]
    ds = ds.set_coords(new_coords_vars)
    return ds

In [10]:
sdrive = 'nctemp/'
tdrive = '/h28/naomi/zarr-minimal/'
string = '_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn'
tpath = 'CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/'
#string = '_6hrLev_IPSL-CM6A-LR_ssp585_r1i1p1f1_gr'
#tpath = 'ScenarioMIP/IPSL/IPSL-CM6A-LR/ssp585/r1i1p1f1/6hrLev/'
tgrid = '/gn'
#odir = {'hus':'/h40','ta':'/h7','ua':'/h7','va':'/h7'}
fix_string = '/usr/bin/ncrename -d .deptht\,olevel -v .deptht\,olevel -d .deptht_bounds\,olevel_bounds -v .deptht_bounds\,olevel_bounds '

In [11]:
table_id = 'Omon'

for variable_id in ['chl']: #,'dissic']:
    print(sdrive + variable_id + string + '_*.nc')
    
    gfiles = sorted(glob(sdrive + variable_id + string + '_*.nc'))
    print(gfiles)
    for gfile in gfiles:
        os.system(f'{fix_string} {gfile}')
        
    zbdir = tdrive + tpath + variable_id + tgrid

    nc_size = os.path.getsize(gfiles[0])
    ds = xr.open_dataset(gfiles[0])
    svar = ds.variable_id
    nt = ds[svar].shape[0]

    chunksize_optimal = 2e8
    chunksize = max(int(nt*chunksize_optimal/nc_size),1)
    #chunksize = min(chunksize,nt)

    print('nt:',nt,'netcdf size:', nc_size/1e6, 'Mb')
    print('suggested chunksize:', chunksize)

    df7 = xr.open_mfdataset(gfiles, preprocess=set_bnds_as_coords, data_vars='minimal', chunks={'time': chunksize}, 
                      use_cftime=True, combine='by_coords', concat_dim='time')
    print(df7.time.values[0],df7.time.values[-1])

    if 'time' in ds.coords:
        year = sorted(list(set(df7.time.dt.year.values)))    
        print(np.diff(year).sum(), len(year))
        if '3hr' in table_id:
            assert (np.diff(year).sum() == len(year)-1) | (np.diff(year).sum() == len(year)-2)
        elif 'dec' in table_id:
            assert (np.diff(year).sum()/10 == len(year)) | (np.diff(year).sum()/10 == len(year)-1)
        else:
            assert np.diff(year).sum() == len(year)-1

    dsl = xr.open_dataset(gfiles[0])
    tracking_id = dsl.tracking_id
    if len(gfiles) > 1:
        for file in gfiles[1:]:
            dsl = xr.open_dataset(file)
            tracking_id = tracking_id+'\n'+dsl.tracking_id
    df7.attrs['tracking_id'] = tracking_id

    date = str(datetime.datetime.now().strftime("%Y-%m-%d"))
    nstatus = date + ';created; by gcs.cmip6.ldeo@gmail.com'
    df7.attrs['status'] = nstatus

    df7 = df7.chunk(chunks={'time' : chunksize})   # yes, do it again
    #del df7[svar].encoding['missing_value']
    print(zbdir)
    df7.to_zarr(zbdir, consolidated=True, mode='w') 
    df7.attrs['tracking_id']

nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_*.nc
['nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-194912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_195001-204912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_205001-214912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-224912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_225001-234912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_235001-244912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-254912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_255001-264912.nc', 'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_265001-274912.nc']
nt: 1200 netcdf size: 14031.640174 Mb
suggested chunksize: 17
1850-01-16 12:00:00 2749-12-16 12:00:00
899 900


/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/

/h28/naomi/zarr-minimal/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/chl/gn


In [5]:
assert False

AssertionError: 

In [7]:
import xarray as xr
#gfiles = ['nctemp/chl_Oyr_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_1850-2149.nc', 
#          'nctemp/chl_Oyr_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_2150-2449.nc', 
#          'nctemp/chl_Oyr_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_2450-2749.nc']
gfiles = ['nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-194912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_195001-204912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_205001-214912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-224912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_225001-234912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_235001-244912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-254912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_255001-264912.nc', 
           'nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_265001-274912.nc']
for gfile in gfiles:
    print(gfile)
    ds = xr.open_dataset(gfile,engine='h5netcdf')
    print(ds.coords)

nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-194912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * deptht   (deptht) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 1949-12-16T12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_195001-204912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * deptht   (deptht) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) datetime64[ns] 1950-01-16T12:00:00 ... 2049-12-16T12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_205001-214912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * deptht   (deptht) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) datetime64[ns] 2050-01-16T12:00:00 ... 2149-12-16T12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-224912.nc
Coordinates:
    nav_lat  (y, x)

/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/xarray/coding/times.py:459: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/

Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * olevel   (olevel) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) object 2250-01-16 12:00:00 ... 2349-12-16 12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_235001-244912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * olevel   (olevel) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) object 2350-01-16 12:00:00 ... 2449-12-16 12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-254912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * olevel   (olevel) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time     (time) object 2450-01-16 12:00:00 ... 2549-12-16 12:00:00
nctemp/chl_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_255001-264912.nc
Coordinates:
    nav_lat  (y, x) float32 ...
    nav_lon  (y, x) float32 ...
  * olevel   (olevel) float32 0.50576 1.5558553 ..

In [ ]:
import os 
nc_size = os.path.getsize(gfiles[0])
ds = xr.open_dataset(gfiles[0])
svar = ds.variable_id
nt = ds[svar].shape[0]

chunksize_optimal = 2e8
chunksize = max(int(nt*chunksize_optimal/nc_size),1)
#chunksize = min(chunksize,nt)

print('nt:',nt,'netcdf size:', nc_size/1e6, 'Mb')
print('suggested chunksize:', chunksize)

df7 = xr.open_mfdataset(gfiles, preprocess=set_bnds_as_coords, data_vars='minimal', chunks={'time': chunksize}, 
                  use_cftime=True, combine='by_coords', concat_dim='time')
